In [80]:
import pandas as pd
import numpy as np


In [61]:
log_data = pd.read_csv('/Users/RICARD/Downloads/log_fw_4.csv', sep=';', header=None)

In [62]:
log_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17674091 entries, 0 to 17674090
Data columns (total 11 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       object 
 1   1       object 
 2   2       object 
 3   3       object 
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       object 
 8   8       object 
 9   9       float64
 10  10      float64
dtypes: float64(5), object(6)
memory usage: 1.4+ GB


In [63]:
log_data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,2022-02-23 09:30:55,89.248.165.121,17.17.17.17,TCP,50242.0,25918.0,999.0,DENY,eth0,NaN,6.0
1,2022-02-23 09:30:57,159.223.115.197,17.17.17.17,TCP,47162.0,24474.0,999.0,DENY,eth0,NaN,6.0
2,2022-02-23 09:31:03,89.248.165.121,17.17.17.17,TCP,50242.0,23937.0,999.0,DENY,eth0,NaN,6.0
3,2022-02-23 09:31:09,91.240.118.73,17.17.17.17,TCP,41839.0,1996.0,999.0,DENY,eth0,NaN,6.0
4,2022-02-23 09:31:12,51.222.253.20,17.17.17.17,TCP,46712.0,443.0,1.0,PERMIT,eth0,NaN,6.0


In [64]:
log_data[3].value_counts()

TCP    17673790
Name: 3, dtype: int64

In [65]:
def to_datetime(x):
    return pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S.%f')

def prepare_log_data(log_data):
    log_data.columns = ['timestamp', 'IPsrc', 'IPdst', 'proto', 'portsrc', 'portdst', 'rule','action','7', '8','9']
    log_data.timestamp = to_datetime(log_data.timestamp)
    log_data.portdst = log_data.portdst.astype('object')
    return log_data

In [66]:
data = prepare_log_data(log_data)

In [67]:
data.pop('8')

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
            ..
17674086   NaN
17674087   NaN
17674088   NaN
17674089   NaN
17674090   NaN
Name: 8, Length: 17674091, dtype: float64

In [68]:
data.head(10)

,timestamp,IPsrc,IPdst,proto,portsrc,portdst,rule,action,7,9
0,2022-02-23 09:30:55,89.248.165.121,17.17.17.17,TCP,50242.0,25918.0,999.0,DENY,eth0,6.0
1,2022-02-23 09:30:57,159.223.115.197,17.17.17.17,TCP,47162.0,24474.0,999.0,DENY,eth0,6.0
2,2022-02-23 09:31:03,89.248.165.121,17.17.17.17,TCP,50242.0,23937.0,999.0,DENY,eth0,6.0
3,2022-02-23 09:31:09,91.240.118.73,17.17.17.17,TCP,41839.0,1996.0,999.0,DENY,eth0,6.0
4,2022-02-23 09:31:12,51.222.253.20,17.17.17.17,TCP,46712.0,443.0,1.0,PERMIT,eth0,6.0
5,2022-02-23 09:31:38,89.248.165.121,17.17.17.17,TCP,50242.0,18383.0,999.0,DENY,eth0,6.0
6,2022-02-23 09:31:47,89.248.165.121,17.17.17.17,TCP,50242.0,19409.0,999.0,DENY,eth0,6.0
7,2022-02-23 09:31:54,193.3.53.11,17.17.17.17,TCP,52274.0,2001.0,999.0,DENY,eth0,6.0
8,2022-02-23 09:31:55,112.85.42.15,17.17.17.17,TCP,39666.0,22.0,4.0,PERMIT,eth0,6.0
9,2022-02-23 09:31:55,112.85.42.15,17.17.17.17,TCP,39666.0,22.0,8.0,PERMIT,eth0,6.0


In [69]:
data.portsrc = data.portsrc.astype('object')

In [70]:
data.dtypes

timestamp    datetime64[ns]
IPsrc                object
IPdst                object
proto                object
portsrc              object
portdst              object
rule                float64
action               object
7                    object
9                   float64
dtype: object

In [71]:
data.dropna(inplace=True)

In [72]:
data.head()

,timestamp,IPsrc,IPdst,proto,portsrc,portdst,rule,action,7,9
0,2022-02-23 09:30:55,89.248.165.121,17.17.17.17,TCP,50242.0,25918.0,999.0,DENY,eth0,6.0
1,2022-02-23 09:30:57,159.223.115.197,17.17.17.17,TCP,47162.0,24474.0,999.0,DENY,eth0,6.0
2,2022-02-23 09:31:03,89.248.165.121,17.17.17.17,TCP,50242.0,23937.0,999.0,DENY,eth0,6.0
3,2022-02-23 09:31:09,91.240.118.73,17.17.17.17,TCP,41839.0,1996.0,999.0,DENY,eth0,6.0
4,2022-02-23 09:31:12,51.222.253.20,17.17.17.17,TCP,46712.0,443.0,1.0,PERMIT,eth0,6.0


# Supervised learning

In [138]:
# get IPsrc, IPdst, portsrc, portdst, action, rule, 9 into a new dataframe  X
X = data[['IPsrc', 'IPdst','portsrc', 'portdst', '9']]

# recode IPsrc and IPdst to numeric values
X['IPsrc'] = X['IPsrc'].astype('category')
X['IPdst'] = X['IPdst'].astype('category')

X['IPsrc'] = X['IPsrc'].cat.codes
X['IPdst'] = X['IPdst'].cat.codes

print(X.head())

print('\n')
y = data['action']
print(y.value_counts(normalize=True))

/var/folders/bn/wmnlmrqn173_fl00j17x5hqm0000gn/T/ipykernel_74948/1425913918.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/bn/wmnlmrqn173_fl00j17x5hqm0000gn/T/ipykernel_74948/1425913918.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/bn/wmnlmrqn173_fl00j17x5hqm0000gn/T/ipykernel_74948/1425913918.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

    IPsrc  IPdst  portsrc  portdst    9
0  346485      0  50242.0  25918.0  6.0
1  107225      0  47162.0  24474.0  6.0
2  346485      0  50242.0  23937.0  6.0
3  348953      0  41839.0   1996.0  6.0
4  291574      0  46712.0    443.0  6.0


PERMIT    0.8469
DENY      0.1531
Name: action, dtype: float64


In [139]:
X.isna().sum()

IPsrc      0
IPdst      0
portsrc    0
portdst    0
9          0
dtype: int64

In [140]:
# Train test split
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [141]:
X_train.shape, X_test.shape

((14139032, 5), (3534758, 5))

In [142]:
y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)

(PERMIT    0.846902
 DENY      0.153098
 Name: action, dtype: float64,
 PERMIT    0.846896
 DENY      0.153104
 Name: action, dtype: float64)

In [143]:
import xgboost as xgb

xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=0)

In [144]:
# convert y categorical variables to 0 and 1
y_train_bin = y_train.map({'DENY': 0, 'PERMIT': 1})
y_test_bin = y_test.map({'DENY': 0, 'PERMIT': 1})


y_train_bin.value_counts(normalize=True), y_test_bin.value_counts(normalize=True)


(1    0.846902
 0    0.153098
 Name: action, dtype: float64,
 1    0.846896
 0    0.153104
 Name: action, dtype: float64)

In [145]:
# make a function to convert object columns to category
def convert_to_float(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype('float64')
    return df

In [146]:
X_train = convert_to_float(X_train)
X_test = convert_to_float(X_test)

In [147]:
X_train.dtypes

IPsrc        int32
IPdst         int8
portsrc    float64
portdst    float64
9          float64
dtype: object

In [148]:
xg_cl.fit(X_train, y_train_bin)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [149]:
pred = xg_cl.predict(X_test)

In [150]:
# get metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy: ', accuracy_score(y_test_bin, pred))
print('Precision: ', precision_score(y_test_bin, pred))
print('Recall: ', recall_score(y_test_bin, pred))
print('F1: ', f1_score(y_test_bin, pred))

# get feature importance
xg_cl.feature_importances_



Accuracy:  1.0
Precision:  1.0
Recall:  1.0
F1:  1.0


array([9.1001153e-04, 0.0000000e+00, 2.2337344e-04, 2.7196592e-01,
       7.2690070e-01], dtype=float32)

In [151]:
# combine feature importance and column names
feature_importance = pd.DataFrame({'feature': X_train.columns, 'importance': xg_cl.feature_importances_})
feature_importance.sort_values(by='importance', ascending=False, inplace=True)
feature_importance


,feature,importance
4,9,0.726901
3,portdst,0.271966
0,IPsrc,0.000910
2,portsrc,0.000223
1,IPdst,0.000000


In [152]:
X['9'].value_counts()

1.0    9387368
6.0    8286422
Name: 9, dtype: int64

# Unsupervised learning

In [133]:
data.head(5)

,timestamp,IPsrc,IPdst,proto,portsrc,portdst,rule,action,7,9
0,2022-02-23 09:30:55,89.248.165.121,17.17.17.17,TCP,50242.0,25918.0,999.0,DENY,eth0,6.0
1,2022-02-23 09:30:57,159.223.115.197,17.17.17.17,TCP,47162.0,24474.0,999.0,DENY,eth0,6.0
2,2022-02-23 09:31:03,89.248.165.121,17.17.17.17,TCP,50242.0,23937.0,999.0,DENY,eth0,6.0
3,2022-02-23 09:31:09,91.240.118.73,17.17.17.17,TCP,41839.0,1996.0,999.0,DENY,eth0,6.0
4,2022-02-23 09:31:12,51.222.253.20,17.17.17.17,TCP,46712.0,443.0,1.0,PERMIT,eth0,6.0


In [137]:
len(data.portdst.unique()), len(data.portsrc.unique()), len(data.rule.unique())

(65534, 64755, 12)

In [154]:
data_sample = data.sample(frac=1000, random_state=0)

data_sample.shape

(1767379, 10)

In [157]:
# group by IPsrc, and count the occurences of each rule

data_sample.groupby(['IPsrc', 'rule']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)



,IPsrc,rule,counts
9172,109.234.162.235,555.0,937970
95576,51.158.121.250,1.0,30902
96705,52.70.240.171,1.0,23538
77557,23.22.35.162,1.0,23237
79867,3.224.220.101,1.0,22387
...,...,...,...
71711,209.141.45.116,999.0,1
71710,209.141.45.116,6.0,1
71709,209.141.44.203,6.0,1
71708,209.141.44.191,5.0,1


In [190]:
# create function of the above
def get_IPsrc_stats(data):
    IPsrc_deny = data[data.action == 'DENY'].groupby('IPsrc')['IPdst'].count().sort_values(ascending=False)
    IPsrc_deny = pd.DataFrame([IPsrc_deny.index,IPsrc_deny]).transpose()
    IPsrc_deny.columns = ['IPsrc', 'deny_count']
    mean_timestamp_byIP = data.groupby('IPsrc').timestamp.mean().reset_index(name='mean_timestamp').sort_values(by='mean_timestamp', ascending=False)
    IPsrc_counts = data.groupby('IPsrc').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
    return IPsrc_deny, mean_timestamp_byIP, IPsrc_counts

# create function to create df_supervised with IPsrc_deny, mean_timestamp_byIP, IPsrc_counts

def create_unsupervised_df(data):
    IPsrc_deny, mean_timestamp_byIP, IPsrc_counts = get_IPsrc_stats(data)
    ratio = pd.merge(IPsrc_counts, IPsrc_deny, on='IPsrc')
    ratio['ratio_deny_total'] = ratio['deny_count'] / ratio['counts']
    df_supervised = pd.merge(IPsrc_counts, mean_timestamp_byIP, on='IPsrc')
    df_supervised = pd.merge(df_supervised, ratio[['IPsrc','ratio_deny_total']], on='IPsrc')
    return df_supervised



In [191]:
unsupervised_df = create_unsupervised_df(data_sample)

unsupervised_df.head()

,IPsrc,counts,mean_timestamp,ratio_deny_total
0,45.227.253.99,7419,2022-11-26 03:47:28.093139200,0.999865
1,198.46.189.193,6202,2023-02-28 12:34:50.847307520,0.972267
2,79.124.62.130,4691,2022-08-06 03:59:21.698785024,0.996376
3,79.124.62.82,4503,2022-08-05 20:52:37.032644864,0.996669
4,79.124.62.78,4460,2022-08-05 19:21:47.252242176,0.998206
